In [ ]:
import cv2
import numpy as np
from collections import deque
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def ask_for_tracker():
    print("Выберите API-интерфейс трекера для использования:")
    print("Нажмите 1 для MIL")
    print("Нажмите 2 для KCF")
    print("Нажмите 3 для CSTR")

    choice = input("Введите число от 1 до 3: ")
    if choice == '1':
        tracker=cv2.TrackerMIL_create()
    if choice == '2':
        tracker=cv2.TrackerKCF_create()
    if choice == '3':
        tracker=cv2.TrackerCSRT_create()

    return tracker

In [ ]:
tracker = ask_for_tracker()

Выберите API-интерфейс трекера для использования:
Нажмите 1 для MIL
Нажмите 2 для KCF
Нажмите 3 для CSTR
Введите число от 1 до 3: 3


In [ ]:
tracker_name = str(tracker).split()[0][1:]

In [ ]:
cap = cv2.VideoCapture("MyDrive/OpenCV/citytraffic.mp4")

In [ ]:
ret , frame = cap.read()

In [ ]:
import cv2

roi = cv2.selectROI(frame,False)

ret = tracker.init(frame,roi)

pts = deque(maxlen=124)

In [ ]:

start_time = time.time()
i = 0
while True:
    if (i==100):
        print("--- %s seconds ---" % (time.time() - start_time))

    ret, frame = cap.read()

    success,roi = tracker.update(frame)

    (x,y,w,h) = tuple(map(int,roi))

    if success:

        pts1 = (x,y)
        pts2 = (x+w,y+h)
        cv2.rectangle(frame,pts1,pts2,(255,125,25),3)
        center=(int(x+w/2),int(y+h/2))
        pts.appendleft(center)
        for j in range(1,len(pts)):
            if pts[j-1] is None or pts[j] is None:
                continue
            thickness = int(np.sqrt(64 / float(j + 1)) * 2.5)
            cv2.line(frame, pts[j - 1], pts[j], (0, 0, 255), thickness)

    else:

        cv2.putText(frame,"Fail to track object",(100,200),cv2.FONT_HERSHEY_SIMPLEX,1,(25,125,255),3)

    cv2.putText(frame,tracker_name,(20,400),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),3)

    cv2.imshow(tracker_name,frame)

    if cv2.waitKey(50)&0xFF==27:
        break
    i+=1

In [ ]:
cap.release()
cv2.destroyAllWindows()